<a href="https://colab.research.google.com/github/alorber/MAML-Implementation/blob/main/MAML_Implementation_FINAL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Midterm Project
## Implementation of [*Model-Agnostic Meta Learning for Fast Adaptation of Deep Network* by Finn](https://arxiv.org/abs/1703.03400)
### Andrew Lorber and Mark Koszykowski

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from time import process_time
from google.colab import files

## Sinusoid class
Amplitude varies within [0.1, 5.0]

Phase varies within [0, π]

In [ ]:
# Creates a sinusoid from the given distribution
class SineWave:
    def __init__(self):
        self.amplitude = np.random.uniform(low=0.1, high=5.0)
        self.phase = np.random.uniform(0, np.pi)
        
    # returns k samples from the sine wave
    # x is sampled uniformly from [-5.0, 5.0]
    def sample(self, k):
        x = tf.random.uniform(shape=(k,1), minval=-5.0, maxval=5.0)
        y = self.amplitude * (tf.sin(x - self.phase))
        return x,y

    # Returns 1000 points of sinewave for graph
    def truth(self):
        x = tf.linspace(-5.0, 5.0, 1000)
        y = self.amplitude * (tf.sin(x - self.phase))
        return tf.reshape(x, (1000,1)),y

## MAML Class
*Subclass of the Keras Model Class*

---

The model consists of two hidden layers of size 40 with ReLU nonlinearities.

The class has functions to feed forward input, meta train with MAML, and fine-tune to a specific sinusoid.

In [ ]:
# MAML Class
class MAML(keras.Model):
    # Builds the model as defined in the paper: 2 hidden layers of size 40 with ReLU
    def __init__(self, k=10, alpha=0.01, beta=0.01):
        super().__init__()

        # Defines layers of model
        self.hidden_layer_1 = keras.layers.Dense(40, input_shape=(1,))
        self.hidden_layer_2 = keras.layers.Dense(40)
        self.output_layer = keras.layers.Dense(1)

        # Sets optimizer and loss function
        self.optimizer = keras.optimizers.Adam(learning_rate=beta)
        self.loss_fnc = keras.losses.mean_squared_error
        
        self.k = k
        self.alpha = alpha

    # Passes input, x, through the neural network and returns output
    def f(self, x):
        x_h_2 = keras.activations.relu(self.hidden_layer_1(x))
        x_out = keras.activations.relu(self.hidden_layer_2(x_h_2))
        y = self.output_layer(x_out)
        return y

    # Trains model following Algorithm 2 of paper
    def meta_train(self, epochs=10000, tasks = 25):
        num_epochs = epochs 
        num_tasks = tasks   # Number of tasks per epoch
        total_loss = []     # Log of losses for graphing

        # Step 1) Randomly initialize theta
        # Completed during initial forward pass

        # Calculates time per 100 epochs
        t1 = process_time()

        # Step 2) For each epoch do:
        for epoch in range(num_epochs):
            # Track progress
            if epoch % 100 == 0 and epoch != 0:
                t2 = process_time()
                print(f"Meta Learning Epoch: {epoch} - Time Per 100 Epochs: {t2 - t1} seconds")
                t1 = process_time()

            losses = []     # Contains losses of each task for this epoch

            # Step 3) Sample batch of tasks T_i ~ p(T)
            sine_tasks = [SineWave() for _ in range(num_tasks)]

            # Step 4) For each T_i do:
            with tf.GradientTape() as outerTape:
                for sinusoid in sine_tasks:
                    # Step 5) Sample K datapoints D from T_i
                    x_train, y_train = sinusoid.sample(self.k)

                    # Step 6) Evaluate gradient w.r.t theta of Loss(f_theta)
                    with tf.GradientTape() as innerTape:
                        y_hat = self.f(x_train)
                        loss = tf.reduce_mean(self.loss_fnc(y_train, y_hat))
                    gradient = innerTape.gradient(loss, self.trainable_variables)

                    # Step 7) Compute adapted parameters thetas_prime
                    # Want to do this on copy of the model, so original thetas don't change
                    model_copy = MAML()
                    # Can't "set_weights" until weights are initialized (happens on first forward pass)
                    model_copy.f(x_train)   
                    model_copy.set_weights(self.get_weights())
                    
                    # Tensorflow does not allow nested "apply_gradients", 
                    #   so we used the following link for help:
                    # https://stackoverflow.com/questions/58856784/nested-gradient-tape-in-function-tf2-0
                    
                    grad = 0
                    for layer in range(len(model_copy.layers)):
                        # Calculates adapted parameters w/ gradient descent
                        # theta' = theta - alpha * gradients 
                        model_copy.layers[layer].kernel = tf.subtract(self.layers[layer].kernel, tf.Variable(tf.multiply(self.alpha, gradient[grad])))
                        model_copy.layers[layer].bias = tf.subtract(self.layers[layer].bias, tf.Variable(tf.multiply(self.alpha, gradient[grad+1])))

                        grad += 2                        

                    # Step 8) Sample datapoints D' from T_i for the meta update
                    x_train, y_train = sinusoid.sample(self.k)
                    y_hat = model_copy.f(x_train)
                    loss = tf.reduce_mean(model_copy.loss_fnc(y_train, y_hat))
                    losses.append(loss)
                
                # Step 9) End for
                loss = tf.reduce_sum(losses)
                total_loss.append(loss)
            
            gradient = outerTape.gradient(loss, self.trainable_variables)
            
            # Step 10) Update thetas using D'_i and Loss(T_i)
            self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))
            
        # Step 11) End for
        t2 = process_time()
        print(f"Meta Learning Epoch: {epochs} - Time Per 100 Epochs: {t2 - t1} seconds")
        
        # Plots loss over epochs
        fig = plt.figure()
        plt.plot(range(epochs), total_loss)
        plt.title("Meta-Loss per Epoch")
        fig.show()
        fig.savefig("Loss.pdf", bbox_inches='tight', pad_inches=0.25)
        files.download("Loss.pdf")

    
    # Fine-tunes model (and baseline if given) on randomly generated sine-wave
    def fineTune(self, k, grad_steps=10, baseline = False):
        # Creates new sine-wave & gets data-sets
        wave = SineWave()
        x_test, y_test = wave.sample(k)
        x_truth, y_truth = wave.truth()

        # MAML
        # -----

        # Creates copy of model, so it can be fine-tuned to different sine-waves for testing
        maml_copy = MAML()
        maml_copy.f(x_test)   
        maml_copy.set_weights(self.get_weights())

        # Saves the regression line pre-fine-tuning
        y_preupdate_maml = maml_copy.f(x_truth)

        # Applies "grad_steps" gradient steps
        for i in range(grad_steps):
            # Calculates gradient
            with tf.GradientTape() as tape:
                y_hat = maml_copy.f(x_test)
                loss = tf.reduce_mean(maml_copy.loss_fnc(y_test, y_hat))
            gradient = tape.gradient(loss, maml_copy.trainable_variables)
            maml_copy.optimizer.apply_gradients(zip(gradient, maml_copy.trainable_variables))

            # Saves regression line after 1st gradient step
            if(i == 0):
                y_1grad_maml = maml_copy.f(x_truth)
        
        # Saves regression line after "grad_steps" gradient steps
        y_ngrad_maml = maml_copy.f(x_truth)

        # Baseline
        # ---------

        if(baseline):
            # Creates copy of model, so it can be fine-tuned to different sine-waves for testing
            baseline_copy = Baseline()
            baseline_copy.f(x_test)   
            baseline_copy.set_weights(baseline.get_weights())

            # Saves the regression line pre-fine-tuning
            y_preupdate_baseline = baseline_copy.f(x_truth)

            # Applies "grad_steps" gradient steps
            for i in range(grad_steps):
                # Calculates gradient
                with tf.GradientTape() as tape:
                    y_hat = baseline_copy.f(x_test)
                    loss = tf.reduce_mean(baseline_copy.loss_fnc(y_test, y_hat))
                gradient = tape.gradient(loss, baseline_copy.trainable_variables)
                baseline_copy.optimizer.apply_gradients(zip(gradient, baseline_copy.trainable_variables))

                # Saves regression line after 1st gradient step
                if(i == 0):
                    y_1grad_baseline = baseline_copy.f(x_truth)
            
            # Saves regression line after "grad_steps" gradient steps
            y_ngrad_baseline = baseline_copy.f(x_truth)


        # Plots

        # MAML
        # -----

        fig = plt.figure()
        plt.plot(x_truth, y_truth, 'r', x_truth, y_preupdate_maml, 'g:', 
                 x_truth, y_1grad_maml, 'b--', x_truth, y_ngrad_maml, 'm--', 
                 x_test, y_test, 'c^')
        plt.legend(["True Sine-Wave", "Pre-update", "1 Gradient Step", 
                   f"{grad_steps} Gradient Steps", f"{k} Points Used for Gradient"], 
                   loc="upper center", bbox_to_anchor= (0.5,-0.07), ncol=3)
        plt.title(f"MAML, K ={k}")
        fig.show()
        fig.savefig(f"MAML{k}.pdf", bbox_inches='tight', pad_inches=0.25)
        files.download(f"MAML{k}.pdf")

        # Baseline
        # ---------
        
        if(baseline):
            fig = plt.figure()
            plt.plot(x_truth, y_truth, 'r', x_truth, y_preupdate_baseline, 'g:', 
                    x_truth, y_1grad_baseline, 'b--', x_truth, y_ngrad_baseline, 'm--', 
                    x_test, y_test, 'c^')
            plt.legend(["True Sine-Wave", "Pre-update", "1 Gradient Step", 
                    f"{grad_steps} Gradient Steps", f"{k} Points Used for Gradient"], 
                    loc="upper center", bbox_to_anchor= (0.5,-0.07), ncol=3)
            plt.title(f"Baseline, K ={k}")
            fig.show()
            fig.savefig(f"Baseline{k}.pdf", bbox_inches='tight', pad_inches=0.25)
            files.download(f"Baseline{k}.pdf")


## Basline Class
*Subclass of the MAML Class*

---

This class will create a baseline to test the MAML model against

In [ ]:
class Baseline(MAML):
    def __init__(self, k=100):
        super().__init__(k)
    
    def train(self, epochs=10000, tasks=25):
        num_epochs = epochs  
        num_tasks = tasks   # Number of tasks per epoch
        total_loss = []     # Log of losses for graphing


        for epoch in range(num_epochs):
            print(epoch)    # Track progress

            # Sample batch of tasks T_i ~ p(T)
            sine_tasks = [SineWave() for _ in range(num_tasks)]

            # For each T_i do:
            for sinusoid in sine_tasks:
                # Sample K datapoints D from T_i
                x_train, y_train = sinusoid.sample(self.k)

                # Evaluate gradient w.r.t theta of Loss(f_theta)
                with tf.GradientTape() as innerTape:
                    y_hat = self.f(x_train)
                    loss = tf.reduce_mean(self.loss_fnc(y_train, y_hat))
                total_loss.append(loss)
                gradient = innerTape.gradient(loss, self.trainable_variables)

                # Update weights
                self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))


        # Plots loss over epochs
        fig = plt.figure()
        plt.plot(range(epochs*tasks), total_loss)
        plt.title("Loss per Epoch")
        fig.show()
        fig.savefig("Baseline_Loss.pdf", bbox_inches='tight', pad_inches=0.25)
        files.download("Baseline_Loss.pdf")